In [33]:
import requests

# Define parameters
api_key = "2A2D4632-39914B94-362C0EED-0B6EEC33-CE1CBE30-993264B5-9104C8F7-1FD72F8A"
category_id = 1275  # The ID for 'medizinisch'
format = "json"
language = "de"
count = 100  # Number of places to retrieve per request (up to 100)
bbox = 
radius = 50000  # Radius of the search area in meters (10 km in this case)
data = "location"
# Make the API request to get the places list
url = f"http://api.wikimapia.org/?function=place.getbyarea&key={api_key}&&bbox{lat}&category={category_id}&format={format}&language={language}&data_blocks={data}&count={count}&radius={radius}"
response = requests.get(url)
print(response.headers)
# Process the response
if response.status_code == 200:
    data = response.json()
    print(data)
#     places = data.get('places', [])  # Extract the list of places
#     if not places:
#         print("No places found in the response.")
#     for place in places:
#         # Accessing the name and other details
#         name = place.get('name')
#         location = place.get('location')
#         description = place.get('description', 'No description')
#         print(f"Name: {name}, Location: {location}, Description: {description}")
# else:
#     print("Error:", response.status_code)


{'Server': 'nginx', 'Date': 'Fri, 11 Oct 2024 11:19:51 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, OPTIONS', 'Access-Control-Allow-Headers': 'x-requested-with, x-prototype-version, x-request', 'Content-Encoding': 'gzip'}
{'language': 'de', 'places': [{'id': 12399841, 'title': 'Rathenower-Apotheke', 'language_id': 5, 'language': 'de', 'url': 'http://wikimapia.org/12399841/de/Rathenower-Apotheke', 'urlhtml': '<a class="wikimapia-link" href="http://wikimapia.org/12399841/de/Rathenower-Apotheke">Rathenower-Apotheke</a>', 'distance': 1061, 'location': {'north': 52.524858, 'south': 52.5247561, 'east': 13.3574384, 'west': 13.3572561, 'lon': 13.35734725, 'lat': 52.52480705}}, {'id': 13557106, 'title': 'Berliner Medizinhistorisches Museum der Charité', 'language_id': 5, 'language': 'de', 'url': 'http://wikimapia.org/13557106/de/Berliner_Medizinhistor

In [34]:
import requests
import json

# Define the Overpass API endpoint
overpass_url = "http://overpass-api.de/api/interpreter"

# Define the Overpass QL query
overpass_query = """
[out:json][timeout:180];
area["ISO3166-1"="DE"][admin_level=2];
node["amenity"="hospital"](area);
out body;
"""

# Send the POST request to Overpass API
response = requests.post(overpass_url, data={'data': overpass_query})

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    elements = data.get('elements', [])
    
    hospitals = []
    for element in elements:
        tags = element.get('tags', {})
        name = tags.get('name', 'No name')
        address = {
            'street': tags.get('addr:street', ''),
            'housenumber': tags.get('addr:housenumber', ''),
            'city': tags.get('addr:city', ''),
            'postcode': tags.get('addr:postcode', '')
        }
        lat = element.get('lat')
        lon = element.get('lon')
        hospitals.append({
            'name': name,
            'address': address,
            'latitude': lat,
            'longitude': lon
        })
    
    # Print the first 10 hospitals
    for hospital in hospitals[:10]:
        print(hospital)
    
    # Optionally, save the data to a JSON file
    with open('germany_hospitals_osm.json', 'w', encoding='utf-8') as f:
        json.dump(hospitals, f, ensure_ascii=False, indent=4)
    
    print(f"Total hospitals retrieved: {len(hospitals)}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)


{'name': 'Kreiskrankenhaus Torgau', 'address': {'street': 'Christianistraße', 'housenumber': '1', 'city': 'Torgau', 'postcode': '04860'}, 'latitude': 51.5615645, 'longitude': 12.984629}
{'name': 'Stiftungskrankenhaus Nördlingen', 'address': {'street': 'Stoffelsberg', 'housenumber': '4', 'city': 'Nördlingen', 'postcode': ''}, 'latitude': 48.8370929, 'longitude': 10.4985631}
{'name': 'Mund-, Kiefer-, Gesichtschirurgie', 'address': {'street': 'Lindwurmstraße', 'housenumber': '2a', 'city': 'München', 'postcode': '80337'}, 'latitude': 48.1310014, 'longitude': 11.5605247}
{'name': 'Dr. von Haunersches Kinderspital', 'address': {'street': 'Lindwurmstraße', 'housenumber': '4', 'city': 'München', 'postcode': '80337'}, 'latitude': 48.1302821, 'longitude': 11.5592053}
{'name': 'Poliklinik Zahnärztliche Prothetik', 'address': {'street': 'Goethestraße', 'housenumber': '70', 'city': 'München', 'postcode': '80336'}, 'latitude': 48.131676, 'longitude': 11.5581961}
{'name': 'Poliklinik Zahnerhaltung un

In [54]:
import overpy
import spacy

# Initialize the Overpass API
api = overpy.Overpass()

# Load the SpaCy German model for NER
# Ensure you have downloaded 'de_core_news_lg' using:
# python -m spacy download de_core_news_lg
nlp = spacy.load("de_core_news_lg")

# Overpass QL query to get all relevant healthcare facilities in Germany
overpass_query = """
[out:json][timeout:180];
area["ISO3166-1"="DE"][admin_level=2];
(
  // Healthcare facilities
  node["healthcare"](area);
  way["healthcare"](area);
  relation["healthcare"](area);
);
out body;
"""

try:
    print("Executing Overpass QL query...")
    # Execute the Overpass query
    result = api.query(overpass_query)
    print("Query executed successfully.")

    # Initialize sets to store unique names
    all_facility_names = set()
    filtered_facility_names = set()
    person_names_removed = set()

    # Function to extract and add names to the set
    def extract_name(element):
        name = element.tags.get("name")
        if name:
            name = name.strip()
            if name and name.lower() != "no name":
                all_facility_names.add(name)

    print("Processing nodes...")
    # Process nodes
    for node in result.nodes:
        extract_name(node)

    print("Processing ways...")
    # Process ways
    for way in result.ways:
        extract_name(way)

    print("Processing relations...")
    # Process relations
    for relation in result.relations:
        extract_name(relation)

    print("Extraction of all facility names complete.")

    # Save all facility names to a text file
    all_output_file = 'all_healthcare_facilities_names.txt'
    print(f"Saving {len(all_facility_names)} unique names to {all_output_file}...")
    with open(all_output_file, 'w', encoding='utf-8') as f:
        for name in sorted(all_facility_names):
            f.write(name + '\n')
    print(f"Saved all healthcare facility names to {all_output_file}.")

    print("Filtering out person names using SpaCy NER...")
    # Function to filter out person names
    def filter_person_names(name):
        doc = nlp(name)
        # Check if any entity in the name is labeled as PERSON ('PER' in SpaCy German model)
        for ent in doc.ents:
            if ent.label_ == "PER":
                return False  # It's a person name
        return True  # It's a facility name

    # Apply the filter to all facility names
    for name in all_facility_names:
        if filter_person_names(name):
            filtered_facility_names.add(name)
        else:
            person_names_removed.add(name)

    print("Filtering complete.")

    # Save the filtered facility names to a text file
    filtered_output_file = 'filtered_healthcare_facilities_names.txt'
    print(f"Saving {len(filtered_facility_names)} filtered names to {filtered_output_file}...")
    with open(filtered_output_file, 'w', encoding='utf-8') as f:
        for name in sorted(filtered_facility_names):
            f.write(name + '\n')
    print(f"Saved filtered healthcare facility names to {filtered_output_file}.")

    # Save the person names that were removed to a separate text file
    person_removed_file = 'person_names_removed.txt'
    print(f"Saving {len(person_names_removed)} person name entries to {person_removed_file}...")
    with open(person_removed_file, 'w', encoding='utf-8') as f:
        for name in sorted(person_names_removed):
            f.write(name + '\n')
    print(f"Saved person name entries to {person_removed_file}.")

    # Compare the two lists and report counts
    total_all = len(all_facility_names)
    total_filtered = len(filtered_facility_names)
    total_removed = len(person_names_removed)

    print("\nSummary of Results:")
    print(f"Total healthcare facility names retrieved: {total_all}")
    print(f"Total unique healthcare facility names after filtering: {total_filtered}")
    print(f"Total person name entries removed: {total_removed}")

except overpy.exception.OverpassTooManyRequests:
    print("Error: Too many requests. Please wait and try again later.")
except overpy.exception.OverpassGatewayTimeout:
    print("Error: Gateway timeout. The server took too long to respond.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Executing Overpass QL query...
Query executed successfully.
Processing nodes...
Processing ways...
Processing relations...
Extraction of all facility names complete.
Saving 62482 unique names to all_healthcare_facilities_names.txt...
Saved all healthcare facility names to all_healthcare_facilities_names.txt.
Filtering out person names using SpaCy NER...
Filtering complete.
Saving 22598 filtered names to filtered_healthcare_facilities_names.txt...
Saved filtered healthcare facility names to filtered_healthcare_facilities_names.txt.
Saving 39884 person name entries to person_names_removed.txt...
Saved person name entries to person_names_removed.txt.

Summary of Results:
Total healthcare facility names retrieved: 62482
Total unique healthcare facility names after filtering: 22598
Total person name entries removed: 39884


search for facilties including the strings: klinik,praxis,arzt,therapie,haus,fach,therapie,medizin,logo,ärzt,zentrum,logie,chirurgie